In [ ]:
# --- iPython Config --- #
from IPython import get_ipython
if 'IPython.extensions.autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')
else:
    get_ipython().run_line_magic('reload_ext', 'autoreload')
%autoreload 2

# --- System and Path --- #
import os
import sys
REPO_PATH = os.path.abspath(os.path.join('..'))
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)
import warnings
warnings.filterwarnings("ignore")

# --- Data Manipulation --- #
import pandas as pd
import numpy as np
from tqdm import tqdm # Progress bar

# --- Financial Data --- #
import yfinance as yf

# --- Modules --- #
from src import *


USD Deposit = 
- THB-USD(+)
  - Deposit.xlsx
- Stock-USD(+)
  - Transactino.xlsx
  - Sell transaction
- USD-Stock(-)
  - Transaction.xlsx
  - Buy transaction
- Dividend (+)
  - Dividends.xlsx

# THB-USD (Inflow +)

In [ ]:
filename = 'DepositUSD.xlsx'
df_thb_usd = pd.read_excel(REPO_PATH+'/data/private/main/'+filename,
                               index_col='Date',
                               parse_dates=True)

In [ ]:
# Filter only relevant with DimeUSD
df_thb_usd = df_thb_usd[
    (df_thb_usd["ToAccount"] == "DimeUSD")
    | (df_thb_usd["FromAccount"] == "DimeUSD")
]

# Amount
df_thb_usd["Amount"] = df_thb_usd.apply(
    lambda x: x["USD Amount"] if x["ToAccount"] == "DimeUSD" else -x["USD Amount"],
    axis=1,
)

# select features
df_thb_usd = df_thb_usd["Amount"]

In [ ]:
def aggregate_intraday_to_daily(df):
    """
    Args:
        df (Series): index is datetime

    Returns:
        DataFrame: index is datetime
    """
    # keep only date
    df = df.reset_index()
    df["Date"] = df["Date"].dt.date

    # aggregate
    df = df.groupby("Date").sum()

    # datetime index
    df.index = pd.to_datetime(df.index)

    return df

df_thb_usd = aggregate_intraday_to_daily(df_thb_usd)
df_thb_usd.columns = ["THB-USD Amount"] # inflow(+)

# Stock-USD (Inflow +)
Sell transaction

In [ ]:
# Read Transactions.xlsx
filename = 'Transactions.xlsx'
df_stock_usd = pd.read_excel(REPO_PATH+'/data/private/main/'+filename,
                             index_col='Date',
                             parse_dates=True)

In [ ]:
# Keep only Sell transactions
df_stock_usd = df_stock_usd[df_stock_usd["Position"] == "Sell"]

# select features
df_stock_usd = df_stock_usd["Total Amount (USD)"]

In [ ]:
df_stock_usd = aggregate_intraday_to_daily(df_stock_usd)

# rename columns
df_stock_usd.columns = ["Stock-USD"] # inflow(+)

# USD-Stock (Outflow -)
Buy transaction in USD

In [ ]:
# Read Transactions.xlsx
filename = 'Transactions.xlsx'
df_usd_stock = pd.read_excel(REPO_PATH+'/data/private/main/'+filename,
                             index_col='Date',
                             parse_dates=True)

In [ ]:
# Keep only Buy transactions
df_usd_stock = df_usd_stock[df_usd_stock["Position"] == "Buy"]

# # Buy with USD
# df_usd_stock = df_usd_stock[
#     df_usd_stock["Total Amount"].apply(lambda x: str(x).startswith("USD"))
# ]

In [ ]:
# select features
df_usd_stock = df_usd_stock["Total Amount (USD)"] # to Series

In [ ]:
df_usd_stock = aggregate_intraday_to_daily(df_usd_stock)

# rename columns
df_usd_stock.columns = ["USD-Stock"]
# outflow(-)
df_usd_stock['USD-Stock'] = -df_usd_stock['USD-Stock']

# Dividend (Inflow +)

In [ ]:
filename = 'Dividends.xlsx'
df_dividends = pd.read_excel(REPO_PATH+'/data/private/main/'+filename,
                             index_col='Date',
                             parse_dates=True)

In [ ]:
# select features
df_dividends = df_dividends["Net Amount"] # USD

# rename series
df_dividends.rename("Dividends-USD", inplace=True)

# Balance

In [ ]:
# merge all dataframes
df_deposit = pd.concat(
    [
        df_thb_usd,
        df_stock_usd,
        df_usd_stock,
        df_dividends,
    ],
    axis=1,
)
df_deposit.sort_index(inplace=True)

In [ ]:
# Inflow(+)/Outflow(-)
df_deposit["Flow"] = df_deposit.sum(axis=1)

# Balance
df_deposit["Balance"] = df_deposit["Flow"].cumsum()
# if negative, set to 0
df_deposit["Balance"] = df_deposit["Balance"].apply(lambda x: max(0, x))

In [ ]:
# Cumulative Deposit (USD)
df_deposit["Cumulative Deposit (USD)"] = df_deposit['THB-USD Amount'].cumsum()
# ffill
df_deposit["Cumulative Deposit (USD)"] = df_deposit["Cumulative Deposit (USD)"].ffill()

In [ ]:
def save_to_csv(df, REPO_PATH=REPO_PATH):

    time_now = pd.Timestamp.now().strftime("%Y%m%d")

    filename = time_now + '-Deposit.csv'
    folderpath = REPO_PATH + '/data/private/csv/'
    filepath = folderpath + filename

    df.to_csv(filepath)

save_to_csv(df_deposit)